# SDI computation for HCP dataset

### SDI Calculation 

In [ ]:
import os
import numpy as np
from scipy.linalg import eigh
import csv

def compute_structural_laplacian(A):
    D = np.diag(np.sum(A, axis=1))
    with np.errstate(divide='ignore'):
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.sum(A, axis=1)))
    D_inv_sqrt[np.isinf(D_inv_sqrt)] = 0
    L = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
    return L

def graph_spectral_phase_randomize(X, eigvecs, seed=None):
    if seed is not None:
        np.random.seed(seed)
    X_hat = X @ eigvecs  # project into graph spectral domain
    T, N = X_hat.shape
    X_surr = np.zeros_like(X_hat)

    for i in range(N):
        fft_coeff = np.fft.fft(X_hat[:, i])
        mag = np.abs(fft_coeff)
        phase = np.angle(fft_coeff)
        num_phases = len(fft_coeff)

        random_phases = np.random.uniform(0, 2*np.pi, num_phases // 2 - 1)
        new_phase = np.copy(phase)
        new_phase[1:num_phases//2] = random_phases
        new_phase[-(num_phases//2)+1:] = -random_phases[::-1]

        new_fft = mag * np.exp(1j * new_phase)
        X_surr[:, i] = np.fft.ifft(new_fft).real

    return X_surr @ eigvecs.T  # back-project into time domain

def compute_SDI_informed(A, X, num_surrogates=100, K_ratio=0.3, seed=None):
    # Preprocess functional data: demean and normalize each node timeseries
    X = X - X.mean(axis=0)
    X = X / (X.std(axis=0) + 1e-10)

    T, N = X.shape
    L = compute_structural_laplacian(A)
    eigvals, eigvecs = eigh(L)

    K = int(np.floor(N * K_ratio))
    if K == 0 or K == N:
        raise ValueError(f"Invalid K: {K} for N={N} and K_ratio={K_ratio}")

    Vlow, Vhigh = eigvecs[:, :K], eigvecs[:, K:]

    N_c_surr, N_d_surr = np.empty((N, num_surrogates)), np.empty((N, num_surrogates))

    for s in range(num_surrogates):
        X_surr = graph_spectral_phase_randomize(X, eigvecs, seed=seed+s)
        X_hat = X_surr @ eigvecs

        X_c = X_hat[:, :K] @ Vlow.T
        X_d = X_hat[:, K:] @ Vhigh.T

        for r in range(N):
            N_c_surr[r, s] = np.linalg.norm(X_c[:, r])
            N_d_surr[r, s] = np.linalg.norm(X_d[:, r])

    SDI = N_d_surr / (N_c_surr + 1e-10)
    mean_SDI = np.mean(SDI, axis=1)
    print(f"Informed SDI stats: mean={mean_SDI.mean():.4f}, std={mean_SDI.std():.4f}")
    return mean_SDI


def main(functional_dir, structural_dir, output_csv_path, num_surrogates=100, K_ratio=0.3, seed=42):
    func_files = {os.path.splitext(f)[0]: os.path.join(functional_dir, f)
                  for f in os.listdir(functional_dir) if f.endswith('.ts')}
    struct_files = {os.path.splitext(f)[0].replace('_5000', ''): os.path.join(structural_dir, f)
                    for f in os.listdir(structural_dir) if f.endswith('.sc')}

    common_patients = set(func_files.keys()) & set(struct_files.keys())
    print(f"Found {len(common_patients)} patients with matching data.")

    results = []
    max_nodes = 0

    for pid in sorted(common_patients):
        func_path = func_files[pid]
        struct_path = struct_files[pid]

        try:
            X = np.loadtxt(func_path)
            A = np.loadtxt(struct_path)

            if X.size == 0 or A.size == 0:
                print(f"Skipping {pid}: empty data")
                continue

            if len(A.shape) != 2 or A.shape[0] != A.shape[1]:
                print(f"Skipping {pid}: Structural matrix not square {A.shape}")
                continue

            if len(X.shape) != 2:
                print(f"Skipping {pid}: Functional data not 2D {X.shape}")
                continue

            T, N_f = X.shape
            N_s = A.shape[0]

            if N_f != N_s:
                if N_s > N_f:
                    A = A[:N_f, :N_f]
                    N_s = N_f
                    print(f"Truncated structural matrix for {pid} from {A.shape[0]} to {N_f} nodes")
                else:
                    print(f"Skipping {pid}: functional nodes > structural nodes (cannot truncate)")
                    continue

            N = min(N_f, N_s)
            if N == 0:
                print(f"Skipping {pid}: zero dimension after truncation")
                continue

            X = X[:, :N]
            A = A[:N, :N]

            sdi = compute_SDI_informed(A, X, num_surrogates=num_surrogates, K_ratio=K_ratio, seed=seed)
            sdi = np.log2(sdi + 1e-10)  # log transform, avoid log(0)

            results.append((pid, sdi))
            max_nodes = max(max_nodes, len(sdi))

        except Exception as e:
            print(f"Error processing {pid}: {e}")

    with open(output_csv_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        header = ['PatientID'] + [f'SDI_Node_{i+1}' for i in range(max_nodes)]
        writer.writerow(header)

        for pid, sdi in results:
            row = [pid] + list(sdi) + [np.nan]*(max_nodes - len(sdi))
            writer.writerow(row)

    print(f"Saved all SDI results to {output_csv_path}")


if __name__ == "__main__":
    functional_dir = "/Users/arnavkarnik/Documents/Classification/Time_Series_HCP/REST2_RL"
    structural_dir = "/Users/arnavkarnik/Documents/Classification/SC_Connectomes_HCP/SC_yeo_100"
    output_csv_path = "/Users/arnavkarnik/Documents/Classification/results_HCP/all_patients_SDI_informed_normalized_hcp.csv"
    main(functional_dir, structural_dir, output_csv_path, num_surrogates=100, K_ratio=0.3, seed=42)

In [ ]:
import os
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import csv

def plot_matrix(mat, title, cmap='viridis'):
    plt.figure(figsize=(5, 5))
    plt.imshow(mat, cmap=cmap)
    plt.colorbar()
    plt.title(title)
    plt.tight_layout()
    plt.show()

def normalize_matrix(mat):
    min_val = np.min(mat)
    max_val = np.max(mat)
    if max_val - min_val == 0:
        return np.zeros_like(mat)
    return (mat - min_val) / (max_val - min_val)

def compute_structural_laplacian(A):
    D = np.diag(np.sum(A, axis=1))
    with np.errstate(divide='ignore'):
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.sum(A, axis=1)))
    D_inv_sqrt[np.isinf(D_inv_sqrt)] = 0
    L = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
    return L

def graph_spectral_phase_randomize(X, eigvecs, seed=None):
    if seed is not None:
        np.random.seed(seed)
    X_hat = X @ eigvecs  # project into graph spectral domain
    T, N = X_hat.shape
    X_surr = np.zeros_like(X_hat)

    for i in range(N):
        fft_coeff = np.fft.fft(X_hat[:, i])
        mag = np.abs(fft_coeff)
        phase = np.angle(fft_coeff)
        num_phases = len(fft_coeff)

        # randomize phases symmetrically except DC and Nyquist (if even)
        random_phases = np.random.uniform(0, 2*np.pi, num_phases // 2 - 1)
        new_phase = np.copy(phase)
        new_phase[1:num_phases//2] = random_phases
        new_phase[-(num_phases//2)+1:] = -random_phases[::-1]

        new_fft = mag * np.exp(1j * new_phase)
        X_surr[:, i] = np.fft.ifft(new_fft).real

    return X_surr @ eigvecs.T  # back-project into time domain

def energy_based_split(eigvals, energy_threshold=0.5):
    """
    Implements energy-based eigenvector split following Preti and Van De Ville.
    Splits eigenvectors into low and high frequency sets such that
    cumulative energy of low freq <= threshold.
    """
    total_energy = np.sum(eigvals)
    cum_energy = np.cumsum(eigvals)
    K = np.searchsorted(cum_energy / total_energy, energy_threshold) + 1
    return K

def compute_SDI_informed(A, X, num_surrogates=100, energy_threshold=0.5, seed=None):
    # Normalize functional data (demean + std)
    X = X - X.mean(axis=0)
    X = X / (X.std(axis=0) + 1e-10)

    T, N = X.shape
    L = compute_structural_laplacian(A)
    eigvals, eigvecs = eigh(L)

    # Energy-based eigenvector split
    K = energy_based_split(eigvals, energy_threshold=energy_threshold)
    # print(f"Energy split at K={K} (energy threshold={energy_threshold})")

    Vlow, Vhigh = eigvecs[:, :K], eigvecs[:, K:]

    N_c_surr, N_d_surr = np.empty((N, num_surrogates)), np.empty((N, num_surrogates))

    for s in range(num_surrogates):
        X_surr = graph_spectral_phase_randomize(X, eigvecs, seed=seed+s if seed else None)
        X_hat = X_surr @ eigvecs

        X_c = X_hat[:, :K] @ Vlow.T
        X_d = X_hat[:, K:] @ Vhigh.T

        for r in range(N):
            N_c_surr[r, s] = np.linalg.norm(X_c[:, r])
            N_d_surr[r, s] = np.linalg.norm(X_d[:, r])

    SDI = N_d_surr / (N_c_surr + 1e-10)
    mean_SDI = np.mean(SDI, axis=1)

    # Min-max normalize SDI between 0 and 1 per patient
    min_val, max_val = np.min(mean_SDI), np.max(mean_SDI)
    if max_val - min_val > 0:
        mean_SDI_norm = (mean_SDI - min_val) / (max_val - min_val)
    else:
        mean_SDI_norm = np.zeros_like(mean_SDI)

    return mean_SDI_norm, K

def main(functional_dir, structural_dir, output_csv_path, num_surrogates=100, energy_threshold=0.5, seed=42):
    func_files = {os.path.splitext(f)[0]: os.path.join(functional_dir, f)
                  for f in os.listdir(functional_dir) if f.endswith('.ts')}
    struct_files = {os.path.splitext(f)[0].replace('_5000', ''): os.path.join(structural_dir, f)
                    for f in os.listdir(structural_dir) if f.endswith('.sc')}

    common_patients = set(func_files.keys()) & set(struct_files.keys())
    print(f"Found {len(common_patients)} patients with matching data.")

    results = []
    max_nodes = 0

    # For plotting examples
    example_plotted = 0
    max_examples = 3

    for pid in sorted(common_patients):
        func_path = func_files[pid]
        struct_path = struct_files[pid]

        try:
            X = np.loadtxt(func_path)
            A = np.loadtxt(struct_path)

            print(f"Patient {pid}: Functional shape {X.shape}, Structural shape {A.shape}")

            if X.size == 0 or A.size == 0:
                print(f"Skipping {pid}: empty data")
                continue

            if len(A.shape) != 2 or A.shape[0] != A.shape[1]:
                print(f"Skipping {pid}: Structural matrix not square {A.shape}")
                continue

            if len(X.shape) != 2:
                print(f"Skipping {pid}: Functional data not 2D {X.shape}")
                continue

            T, N_f = X.shape
            N_s = A.shape[0]

            # Truncate to min dimension to fix mismatch
            N = min(N_f, N_s)
            if N == 0:
                print(f"Skipping {pid}: zero dimension after truncation")
                continue

            # Truncate matrices
            X = X[:, :N]
            A = A[:N, :N]

            # Normalize structural matrix between 0 and 1
            A_norm = normalize_matrix(A)

            # Plot first few patients
            if example_plotted < max_examples:
                print(f"Plotting matrices for patient {pid}")
                plot_matrix(X, f"Functional data (fMRI) - Patient {pid}", cmap='coolwarm')
                plot_matrix(A_norm, f"Structural Connectivity (normalized) - Patient {pid}")
                example_plotted += 1

            sdi_norm, K = compute_SDI_informed(A_norm, X, num_surrogates=num_surrogates,
                                              energy_threshold=energy_threshold, seed=seed)

            results.append((pid, sdi_norm))
            max_nodes = max(max_nodes, len(sdi_norm))

        except Exception as e:
            print(f"Error processing {pid}: {e}")

    # Save results to CSV
    with open(output_csv_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        header = ['PatientID'] + [f'SDI_Node_{i+1}' for i in range(max_nodes)]
        writer.writerow(header)

        for pid, sdi in results:
            row = [pid] + list(sdi) + [np.nan]*(max_nodes - len(sdi))
            writer.writerow(row)

    print(f"Saved all SDI results to {output_csv_path}")

if __name__ == "__main__":
    functional_dir = "/Users/arnavkarnik/Documents/Classification/Time_Series_HCP/REST2_RL"
    structural_dir = "/Users/arnavkarnik/Documents/Classification/SC_Connectomes_HCP/SC_yeo_100"
    output_csv_path = "/Users/arnavkarnik/Documents/Classification/results_HCP/all_patients_SDI_informed_normalized_hcp.csv"

    main(functional_dir, structural_dir, output_csv_path,
         num_surrogates=100,
         energy_threshold=0.5,
         seed=42)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

# Path to your SDI results CSV
csv_path = "/Users/arnavkarnik/Documents/Classification/results_HCP/all_patients_SDI_informed_normalized_hcp.csv"

# Load SDI data from CSV (skip header)
patient_ids = []
sdi_data = []

with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)  # Skip header
    for row in reader:
        patient_ids.append(row[0])
        sdi_vals = [float(x) if x != '' else np.nan for x in row[1:]]
        sdi_data.append(sdi_vals)

sdi_array = np.array(sdi_data)  # shape: (num_patients, num_nodes)

# Replace empty/nan with np.nan explicitly
sdi_array = np.where(np.isnan(sdi_array), np.nan, sdi_array)

# Plot 1: Histogram of all SDI values across all nodes and patients
plt.figure(figsize=(12,6))
plt.hist(sdi_array[~np.isnan(sdi_array)].flatten(), bins=90, color='c', alpha=0.7)
plt.title("Histogram of all SDI values across all nodes and patients")
plt.xlabel("SDI value")
plt.ylabel("Frequency")
plt.show()

# Plot 2: Mean ± Std Dev of SDI per brain node across patients
mean_sdi = np.nanmean(sdi_array, axis=0)
std_sdi = np.nanstd(sdi_array, axis=0)
nodes = np.arange(1, len(mean_sdi)+1)

plt.figure(figsize=(14,6))
plt.errorbar(nodes, mean_sdi, yerr=std_sdi, fmt='-o', ecolor='r', capsize=5)
plt.title("Mean ± Std Dev of SDI per brain node across patients")
plt.xlabel("Brain node")
plt.ylabel("SDI")
plt.grid(True)
plt.show()

# Plot 3: Distribution of total SDI per patient
total_sdi_per_patient = np.nansum(sdi_array, axis=1)

plt.figure(figsize=(10,5))
plt.hist(total_sdi_per_patient, bins=30, color='m', alpha=0.7)
plt.title("Distribution of total SDI scores across patients")
plt.xlabel("Total SDI")
plt.ylabel("Number of patients")
plt.show()

# Plot 4: Heatmap of SDI for first 5 patients (or less if fewer patients)
num_patients_to_show = min(25, sdi_array.shape[0])
plt.figure(figsize=(14,6))
sns.heatmap(sdi_array[:num_patients_to_show], cmap='viridis', xticklabels=np.arange(1, sdi_array.shape[1]+1), yticklabels=patient_ids[:num_patients_to_show])
plt.title("SDI heatmap for first 5 patients (nodes on x-axis)")
plt.xlabel("Brain node")
plt.ylabel("Patient ID")
plt.show()

In [ ]:
import os
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import csv

def plot_matrix(mat, title, cmap='viridis'):
    plt.figure(figsize=(5, 5))
    plt.imshow(mat, cmap=cmap)
    plt.colorbar()
    plt.title(title)
    plt.tight_layout()
    plt.show()

def normalize_matrix(mat):
    min_val = np.min(mat)
    max_val = np.max(mat)
    if max_val - min_val == 0:
        return np.zeros_like(mat)
    return (mat - min_val) / (max_val - min_val)

def compute_structural_laplacian(A):
    D = np.diag(np.sum(A, axis=1))
    with np.errstate(divide='ignore'):
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.sum(A, axis=1)))
    D_inv_sqrt[np.isinf(D_inv_sqrt)] = 0
    L = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
    return L

def graph_spectral_phase_randomize(X, eigvecs, seed=None):
    if seed is not None:
        np.random.seed(seed)
    X_hat = X @ eigvecs  # project into graph spectral domain
    T, N = X_hat.shape
    X_surr = np.zeros_like(X_hat)

    for i in range(N):
        fft_coeff = np.fft.fft(X_hat[:, i])
        mag = np.abs(fft_coeff)
        phase = np.angle(fft_coeff)
        num_phases = len(fft_coeff)

        # randomize phases symmetrically except DC and Nyquist (if even)
        random_phases = np.random.uniform(0, 2*np.pi, num_phases // 2 - 1)
        new_phase = np.copy(phase)
        new_phase[1:num_phases//2] = random_phases
        new_phase[-(num_phases//2)+1:] = -random_phases[::-1]

        new_fft = mag * np.exp(1j * new_phase)
        X_surr[:, i] = np.fft.ifft(new_fft).real

    return X_surr @ eigvecs.T  # back-project into time domain

def energy_based_split(eigvals, energy_threshold=0.5):
    """
    Implements energy-based eigenvector split following Preti and Van De Ville.
    Splits eigenvectors into low and high frequency sets such that
    cumulative energy of low freq <= threshold.
    """
    total_energy = np.sum(eigvals)
    cum_energy = np.cumsum(eigvals)
    K = np.searchsorted(cum_energy / total_energy, energy_threshold) + 1
    return K

def compute_SDI_informed(A, X, num_surrogates=100, energy_threshold=0.5, seed=None):
    # Normalize functional data (demean + std)
    X = X - X.mean(axis=0)
    X = X / (X.std(axis=0) + 1e-10)

    T, N = X.shape
    L = compute_structural_laplacian(A)
    eigvals, eigvecs = eigh(L)

    # Energy-based eigenvector split
    K = energy_based_split(eigvals, energy_threshold=energy_threshold)
    # print(f"Energy split at K={K} (energy threshold={energy_threshold})")

    Vlow, Vhigh = eigvecs[:, :K], eigvecs[:, K:]

    N_c_surr, N_d_surr = np.empty((N, num_surrogates)), np.empty((N, num_surrogates))

    for s in range(num_surrogates):
        X_surr = graph_spectral_phase_randomize(X, eigvecs, seed=seed+s if seed else None)
        X_hat = X_surr @ eigvecs

        X_c = X_hat[:, :K] @ Vlow.T
        X_d = X_hat[:, K:] @ Vhigh.T

        for r in range(N):
            N_c_surr[r, s] = np.linalg.norm(X_c[:, r])
            N_d_surr[r, s] = np.linalg.norm(X_d[:, r])

    SDI = N_d_surr / (N_c_surr + 1e-10)
    mean_SDI = np.mean(SDI, axis=1)

    # Min-max normalize SDI between 0 and 1 per patient
    min_val, max_val = np.min(mean_SDI), np.max(mean_SDI)
    if max_val - min_val > 0:
        mean_SDI_norm = (mean_SDI - min_val) / (max_val - min_val)
    else:
        mean_SDI_norm = np.zeros_like(mean_SDI)

    return mean_SDI_norm, K

def main(functional_dir, structural_dir, output_csv_path, num_surrogates=100, energy_threshold=0.5, seed=42):
    func_files = {os.path.splitext(f)[0]: os.path.join(functional_dir, f)
                  for f in os.listdir(functional_dir) if f.endswith('.ts')}
    struct_files = {os.path.splitext(f)[0].replace('_5000', ''): os.path.join(structural_dir, f)
                    for f in os.listdir(structural_dir) if f.endswith('.sc')}

    common_patients = set(func_files.keys()) & set(struct_files.keys())
    print(f"Found {len(common_patients)} patients with matching data.")

    results = []
    max_nodes = 0

    # For plotting examples
    example_plotted = 0
    max_examples = 3

    for pid in sorted(common_patients):
        func_path = func_files[pid]
        struct_path = struct_files[pid]

        try:
            X = np.loadtxt(func_path)
            A = np.loadtxt(struct_path)

            print(f"Patient {pid}: Functional shape {X.shape}, Structural shape {A.shape}")

            if X.size == 0 or A.size == 0:
                print(f"Skipping {pid}: empty data")
                continue

            if len(A.shape) != 2 or A.shape[0] != A.shape[1]:
                print(f"Skipping {pid}: Structural matrix not square {A.shape}")
                continue

            if len(X.shape) != 2:
                print(f"Skipping {pid}: Functional data not 2D {X.shape}")
                continue

            T, N_f = X.shape
            N_s = A.shape[0]

            # Truncate to max 68 nodes or smaller dimension to fix mismatch
            N = min(68, N_f, N_s)
            if N == 0:
                print(f"Skipping {pid}: zero dimension after truncation")
                continue

            # Truncate matrices
            X = X[:, :N]
            A = A[:N, :N]

            # Normalize structural matrix between 0 and 1
            A_norm = normalize_matrix(A)

            # Plot first few patients
            if example_plotted < max_examples:
                print(f"Plotting matrices for patient {pid}")
                plot_matrix(X, f"Functional data (fMRI) - Patient {pid}", cmap='coolwarm')
                plot_matrix(A_norm, f"Structural Connectivity (normalized) - Patient {pid}")
                example_plotted += 1

            sdi_norm, K = compute_SDI_informed(A_norm, X, num_surrogates=num_surrogates,
                                              energy_threshold=energy_threshold, seed=seed)

            results.append((pid, sdi_norm))
            max_nodes = max(max_nodes, len(sdi_norm))

        except Exception as e:
            print(f"Error processing {pid}: {e}")

    # Save results to CSV
    with open(output_csv_path, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        header = ['PatientID'] + [f'SDI_Node_{i+1}' for i in range(max_nodes)]
        writer.writerow(header)

        for pid, sdi in results:
            row = [pid] + list(sdi) + [np.nan]*(max_nodes - len(sdi))
            writer.writerow(row)

    print(f"Saved all SDI results to {output_csv_path}")

if __name__ == "__main__":
    functional_dir = "/Users/arnavkarnik/Documents/Classification/Time_Series_HCP/REST2_RL"
    structural_dir = "/Users/arnavkarnik/Documents/Classification/SC_Connectomes_HCP/SC_yeo_100"
    output_csv_path = "/Users/arnavkarnik/Documents/Classification/results_HCP/all_patients_SDI_informed_normalized_hcp.csv"

    main(functional_dir, structural_dir, output_csv_path,
         num_surrogates=100,
         energy_threshold=0.5,
         seed=42)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

# Path to your SDI results CSV
csv_path = "/Users/arnavkarnik/Documents/Classification/results_HCP/all_patients_SDI_informed_normalized_hcp.csv"

# Load SDI data from CSV (skip header)
patient_ids = []
sdi_data = []

with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)  # Skip header
    for row in reader:
        patient_ids.append(row[0])
        sdi_vals = [float(x) if x != '' else np.nan for x in row[1:]]
        sdi_data.append(sdi_vals)

sdi_array = np.array(sdi_data)  # shape: (num_patients, num_nodes)

# Replace empty/nan with np.nan explicitly
sdi_array = np.where(np.isnan(sdi_array), np.nan, sdi_array)

# Plot 1: Histogram of all SDI values across all nodes and patients
plt.figure(figsize=(12,6))
plt.hist(sdi_array[~np.isnan(sdi_array)].flatten(), bins=90, color='c', alpha=0.7)
plt.title("Histogram of all SDI values across all nodes and patients")
plt.xlabel("SDI value")
plt.ylabel("Frequency")
plt.show()

# Plot 2: Mean ± Std Dev of SDI per brain node across patients
mean_sdi = np.nanmean(sdi_array, axis=0)
std_sdi = np.nanstd(sdi_array, axis=0)
nodes = np.arange(1, len(mean_sdi)+1)

plt.figure(figsize=(14,6))
plt.errorbar(nodes, mean_sdi, yerr=std_sdi, fmt='-o', ecolor='r', capsize=5)
plt.title("Mean ± Std Dev of SDI per brain node across patients")
plt.xlabel("Brain node")
plt.ylabel("SDI")
plt.grid(True)
plt.show()

# Plot 3: Distribution of total SDI per patient
total_sdi_per_patient = np.nansum(sdi_array, axis=1)

plt.figure(figsize=(10,5))
plt.hist(total_sdi_per_patient, bins=30, color='m', alpha=0.7)
plt.title("Distribution of total SDI scores across patients")
plt.xlabel("Total SDI")
plt.ylabel("Number of patients")
plt.show()

# Plot 4: Heatmap of SDI for first 5 patients (or less if fewer patients)
num_patients_to_show = min(25, sdi_array.shape[0])
plt.figure(figsize=(14,6))
sns.heatmap(sdi_array[:num_patients_to_show], cmap='viridis', xticklabels=np.arange(1, sdi_array.shape[1]+1), yticklabels=patient_ids[:num_patients_to_show])
plt.title("SDI heatmap for first 5 patients (nodes on x-axis)")
plt.xlabel("Brain node")
plt.ylabel("Patient ID")
plt.show()

In [ ]:
from nilearn import plotting, datasets
import nibabel as nib
import numpy as np

# Load the fsaverage template
fsaverage = datasets.fetch_surf_fsaverage()

# Load the left and right hemisphere SDI surface values
sdi_left = nib.load('/Users/arnavkarnik/Documents/Classification/Desikan-Killiany Atlas/23262.L.func.gii').darrays[0].data
sdi_right = nib.load('/Users/arnavkarnik/Documents/Classification/Desikan-Killiany Atlas/23262.R.func.gii').darrays[0].data

# Plot left hemisphere
plotting.plot_surf_stat_map(
    surf_mesh=fsaverage['pial_left'],
    stat_map=sdi_left,
    hemi='left',
    colorbar=True,
    title='Left Hemisphere SDI',
    cmap='coolwarm'
)

# Plot right hemisphere
plotting.plot_surf_stat_map(
    surf_mesh=fsaverage['pial_right'],
    stat_map=sdi_right,
    hemi='right',
    colorbar=True,
    title='Right Hemisphere SDI',
    cmap='coolwarm'
)

plotting.show()
